In [1]:
with open('input.txt', "r") as file:
    data = file.read().strip()

print(len(data))

17566


# Part 1

In [2]:
def split_tiles(tile_str):
    tile = tile_str.split('\n')
    return (int(tile[0].split()[1][:-1]), tile[1:])

tiles = list(map(lambda x: split_tiles(x), data.split('\n\n')))

tiles[0]

(3733,
 ['#.#.#..#.#',
  '...#..#.##',
  '.#...#...#',
  '#.###.....',
  '..##.....#',
  '##......#.',
  '#......#.#',
  '...#....#.',
  '###.##...#',
  '######..##'])

In [3]:
borders_ids = {}
def add_borders_id(border, tile_id):
    cur_ids = borders_ids.get(border, [])
    
    cur_ids.append(tile_id)
    borders_ids[border] = cur_ids


for tile_id, tile in tiles:
    borders = [tile[0], tile[-1], ''.join([line[0] for line in tile]),
               ''.join([line[-1] for line in tile])]
    for border in borders:
        add_borders_id(min(border, border[::-1]), tile_id)

unpaired_counts = {}
for group in borders_ids.values():
    if len(group) == 1:
        for tile_id in group:
            unpaired_counts[tile_id] = unpaired_counts.get(tile_id, 0) + 1

corner_tiles_ids = [tile for tile in unpaired_counts if unpaired_counts[tile] == 2]
corner_tiles_ids

[1621, 3547, 3389, 1657]

In [4]:
from functools import reduce

reduce(lambda a, b: a * b, corner_tiles_ids), corner_tiles_ids

(32287787075651, [1621, 3547, 3389, 1657])

# Part 2

In [5]:
tiles_d = {tile_id: tile_map for tile_id, tile_map in tiles}

In [6]:
def bkey(s):
    return min(s, s[::-1])

def rotate_90(tile_map):
    rotated = list(zip(*reversed(tile_map)))
    rotated = [''.join(line) for line in rotated]
    return rotated

def gen_trasforms(tile_map):
    step = 1
    while True:
        yield tile_map
        tile_map = rotate_90(tile_map)
        if step == 4: # flip
            tile_map = [line[::-1] for line in tile_map]
        elif step == 8:
            break
        step += 1

def list_sides(tile_map):
    s1, s2, s3, s4 = (tile_map[0], ''.join([line[-1] for line in tile_map]),
                      tile_map[-1], ''.join([line[0] for line in tile_map]))
    return s1, s2, s3, s4

def fit_tile(tile_map, left, up):
    '''
    bruteforce the orientation
    s1, s2, s3, s4 ~ N, E, S, W
    '''
    res = None

    step = 0
    for transform in gen_trasforms(tile_map):
        s1, s2, s3, s4 = list_sides(transform)
        if (
            (s1 == up or (up is None and len(borders_ids[bkey(s1)]) == 1))
            and (s4 == left or (left is None and len(borders_ids[bkey(s4)]) == 1))
        ):
            res = transform
            break

    return res


In [7]:
tiles_ids_to_place = list(tiles_d.keys())
tiles_placed = []

pic_line = []
pic = [pic_line]
row_i, col_i = 0, 0
left, up = None, None

next_tile_id = corner_tiles_ids[0]

tile_n = 0
while True:
    tiles_ids_to_place = [tile_id for tile_id in tiles_ids_to_place if tile_id != next_tile_id]
    
    if row_i == 0:
        up = None
    else:
        up = pic[row_i-1][col_i][1][-1]

    if col_i == 0:
        left = None
    else:
        left_tile_map = pic_line[-1][1]
        left = ''.join([line[-1] for line in left_tile_map])
    
    tile_map = fit_tile(tiles_d[next_tile_id], left, up)
    tile_n += 1
    pic_line.append((next_tile_id, tile_map))
    right = ''.join([line[-1] for line in tile_map])
    if not tiles_ids_to_place:
        break
    # find next tile and its place
    # manage pic_line, row_i, col_i
    if len(borders_ids[bkey(right)]) == 1:
        f_tile_id = pic_line[0][0]
        down = pic_line[0][1][-1]
        next_tile_id = [tile_id for tile_id in borders_ids[bkey(down)] if tile_id != f_tile_id][0]
        pic_line = []
        pic.append(pic_line)
        row_i += 1
        col_i = 0
    else:
        next_tile_id = [tile_id for tile_id in borders_ids[bkey(right)] if tile_id != next_tile_id][0]
        col_i += 1
    
    if next_tile_id not in tiles_ids_to_place:
        print('EPC FAIL!!! Bad input data!')
        break

In [8]:
H = len(pic)
W = len(pic[1])

to_draw = []

for row in range(H):
    line = pic[row]
    tile_len = len(line[0][1])
    for sub_row in range(1, tile_len-1):
        big_line = ''.join([tile[sub_row][1:-1] for _, tile in line])
        to_draw.append(big_line)

for line in to_draw:
    print(line)

.........#...#..##..#.......#...#.....###..........##.......#.......#....##.#.##......#.........
..........##..#...#.#.....#..#....#......#...#...........#...#...........#.......#..#..#.#.##..#
.#..#.....#....##...####...........#..#..#..#.#..#..#..#...#.#.........#......#....###.......#..
.##....###..........#...#..##...#.....##......#....#............#...#.....##.#..#........#......
...#...........#.......#....#..#.....#....#...#.....#......#...##.....#.......##..#..#..........
.......#.....##......#..##.#.....#.......##.......#......#..#....##..##....##........#......#...
..#.##.#..#...#..#.#..###.....#..##......#..##..##.......##..............#....#................#
.#............#.........#..........#.#.##...............#............#..#.......##.#...........#
......#.........#..#.#...##...#........#..#..#....#...##...##...#.#......###...##..#.......#.#..
....#.............#...........##.#......#....#.#...#......#.#...#...#......#......#.........#...
..#...#....#.......#..#.......

In [9]:
monster_pattern = '''
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
'''.split('\n')[1:-1]

In [10]:
def find_pattern(pic, pattern):
    # pattern = ''.join(pattern)
    H, W = len(pic), len(pic[0])
    pH, pW = len(pattern), len(pattern[0])

    matches = []

    for y in range(0, H-pH):
        for x in range(0, W-pW):
            match = True
            for dr in range(pH * pW):
                if pattern[dr//pW][dr%pW] != '#':
                    continue
                else:
                    if pic[y + dr//pW][x + dr%pW] != '#':
                        match = False
                        break
            if match:
                matches.append((x, y))
    return matches

m_monsters = 0
for transform in gen_trasforms(monster_pattern):
    n_monsters = len(find_pattern(to_draw, transform))
    if m_monsters < n_monsters:
        m_monsters = n_monsters
    print(sorted(find_pattern(to_draw, transform), key = lambda x: x[0]))

[]
[]
[]
[(1, 74), (3, 31), (11, 20), (13, 50), (19, 27), (23, 5), (23, 56), (29, 42), (29, 73), (37, 39), (45, 39), (57, 61), (64, 56), (69, 15), (69, 64), (74, 40), (75, 12), (80, 50), (85, 23), (90, 28), (90, 71)]
[]
[]
[]
[]


In [11]:
res = ''.join(to_draw).count('#') - ''.join(monster_pattern).count('#') * m_monsters

res

1939